Import libraries


In [11]:
import pandas as pd
import re 

# 06 - NUS Undergraduate Programs


Load the data


In [12]:
ug_program_path = '../backend/data/06 - nus_undergraduate_programmes.csv'
ug_program = pd.read_csv(ug_program_path)

print(ug_program.head())

                                Degree  \
0  Bachelor of Business Administration   
1  Bachelor of Business Administration   
2  Bachelor of Business Administration   
3  Bachelor of Business Administration   
4  Bachelor of Business Administration   

                                     Major  
0               Applied Business Analytics  
1                       Business Economics  
2                                  Finance  
3          Innovation and Entrepreneurship  
4  Leadership and Human Capital Management  


## Entity Extraction


Extract degree amdand major entiies


In [13]:
# Function to extract degree and major entities
def extract_entities(degree, major):
    # Ensure both degree and major are valid strings
    degree_entity = [degree] if isinstance(degree, str) and degree.strip() else []
    major_entity = [major] if isinstance(major, str) and major.strip() else []
    
    return degree_entity, major_entity

# Apply the function to extract degree and major entities
ug_program['degree_entities'], ug_program['major_entities'] = zip(*ug_program.apply(
    lambda row: extract_entities(row['Degree'], row['Major']), axis=1))

# Display the DataFrame with extracted degree and major entities
print(ug_program[['Degree', 'Major', 'degree_entities', 'major_entities']].head())

                                Degree  \
0  Bachelor of Business Administration   
1  Bachelor of Business Administration   
2  Bachelor of Business Administration   
3  Bachelor of Business Administration   
4  Bachelor of Business Administration   

                                     Major  \
0               Applied Business Analytics   
1                       Business Economics   
2                                  Finance   
3          Innovation and Entrepreneurship   
4  Leadership and Human Capital Management   

                         degree_entities  \
0  [Bachelor of Business Administration]   
1  [Bachelor of Business Administration]   
2  [Bachelor of Business Administration]   
3  [Bachelor of Business Administration]   
4  [Bachelor of Business Administration]   

                              major_entities  
0               [Applied Business Analytics]  
1                       [Business Economics]  
2                                  [Finance]  
3          [Innov

## Relationship Extraction


In [20]:
def create_dynamic_relationship(df, from_type, from_id_col, to_type, to_id_col, relationship_type, output_col):
    # List to store formatted relationship dictionaries for each row
    relationship_column = []

    # Iterate through each row of the DataFrame
    for _, row in df.iterrows():
        # Extract the `from_id` and `to_id` values from the specified columns
        from_ids = row[from_id_col] if isinstance(row[from_id_col], list) else [row[from_id_col]]
        to_ids = row[to_id_col] if isinstance(row[to_id_col], list) else [row[to_id_col]]

        # Create a list of dictionaries for each `to_id`
        relationship_dict = [
            {
                "from_type": from_type,
                "from_id": from_id,
                "to_type": to_type,
                "to_id": to_id,
                "type": relationship_type
            }
            for from_id in from_ids if pd.notna(from_id)
            for to_id in to_ids if pd.notna(to_id)  # Only include non-NaN `to_id` values
        ]

        # Append the relationship dictionary or an empty list if no valid `to_id` found
        relationship_column.append(relationship_dict if relationship_dict else [])

    # Add the relationships as a new column to the DataFrame
    df[output_col] = relationship_column
    
    # Return the updated DataFrame with the new relationships column
    return df

from_type = "Major"
from_id_col = "major_entities"  
to_type = "Degree"
to_id_col = "degree_entities"   
relationship_type = "OFFERS"
output_col = "relationship"

# Create relationships for each row and add them to the DataFrame
ug_program_rs = create_dynamic_relationship(ug_program, from_type, from_id_col, to_type, to_id_col, relationship_type, output_col)

# Display the DataFrame with relationships
print(ug_program_rs[['major_entities', 'degree_entities', 'relationship']].head())

                              major_entities  \
0               [Applied Business Analytics]   
1                       [Business Economics]   
2                                  [Finance]   
3          [Innovation and Entrepreneurship]   
4  [Leadership and Human Capital Management]   

                         degree_entities  \
0  [Bachelor of Business Administration]   
1  [Bachelor of Business Administration]   
2  [Bachelor of Business Administration]   
3  [Bachelor of Business Administration]   
4  [Bachelor of Business Administration]   

                                        relationship  
0  [{'from_type': 'Major', 'from_id': 'Applied Bu...  
1  [{'from_type': 'Major', 'from_id': 'Business E...  
2  [{'from_type': 'Major', 'from_id': 'Finance', ...  
3  [{'from_type': 'Major', 'from_id': 'Innovation...  
4  [{'from_type': 'Major', 'from_id': 'Leadership...  


Save updated DF to a new CSV file


In [21]:
ug_program.to_csv('ug_program_final.csv', index=False)